In [1]:
import re
import unicodedata
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer

In [82]:
class Preprocess:
        
        # --------------------------------------- Constructor --------------------------------------- 
        
        def __init__(self,stopword_list):
            self.data_path = ''
            self.stopword_list = stopword_list
                

        # --------------------------------------- Preprocess --------------------------------------- 
        
        def expand_concatenations(self, word):
            
            
            if not re.match('[a-zA-Z]+', word) or re.match('/d+',word):
                for i in range(len(word)):
                    if not('DEVANAGARI ' in unicodedata.name(word[i])):
                        word = word[:i] if( len(word[i:]) < 2 and not word[i:].isnumeric()) else word[:i] + " " + word[i:]
                        break
            else:
                for i in range(len(word)):
                    if ('DEVANAGARI ' in unicodedata.name(word[i])):
                        word = word[i:] if( len(word[:i]) < 2 and not word[:i].isnumeric() ) else word[:i] + " " + word[i:]
                        break

            return(word)
    
        
        def clean_text(self,text: str) -> str:
            try:
                special_chars = r'''!()-[]{};:'"\,<>./?@#$%^&*_~'''
                stemmer = PorterStemmer()
                lemmatizer = WordNetLemmatizer()

                if not(isinstance(text, str)): text = str(text)

                #Removing unprintable characters
                text = ''.join(x for x in text if x.isprintable())

                # Cleaning the urls
                text = re.sub(r'https?://\S+|www\.\S+', '', text)

                # Cleaning the html elements
                text = re.sub(r'<.*?>', '', text)

                # Removing the punctuations
                text = re.sub('[!#?,.:";-@#$%^&*_~<>()-]', '', text)


                # Removing stop words
                text = ' '.join([word for word in text.split() if word not in self.stopword_list])

                # Expanding noisy concatenations (Eg: algorithmआणि  -> algorithm आणि ) 
                text = ' '.join([self.expand_concatenations(word) for word in text.split()])

                preprocessed_text = ""

                for word in text.split(): 
                    if (re.match('\d+', word)):
                        if(word.isnumeric()):
                            preprocessed_text = preprocessed_text + '#N' + " "

                    else:
                        if(re.match('[a-zA-Z]+', word)):
                            if not len(word) < 2:
                                word = word.lower()
    #                             word = lemmatizer.lemmatize(word, pos='v')
                                preprocessed_text = preprocessed_text + word + " "

                        else:
                            preprocessed_text = preprocessed_text + word + " "

                return preprocessed_text
            
            except ValueError as ve:
                print('Error processing:\t',text)
                return ''
        

In [45]:
import pandas as pd
df = pd.read_csv('../Technodifacation/Data/training_data_marathi.csv')

In [86]:
sampletext1 = df['text'].sample().values
pp = Preprocess([])
sampletext2 = 'त्यांना जनतेला पटवून द्यावे लागेल99 '

test_list = ['त्यांना', 'CO2' ,'सीओ२', 'लागेल99', 'Animalत्यांना',
             'त्यांनाAnimal', 'Analogy_त्यांना', 'Science२१', '१२Number', '!@)$&%!#)&$!&$!$', '११.२२', '१','1','11.22','a','B','सी']
# ptext = pp.clean_text(sampletext1)


for text in test_list:
    print(text, '\t--->\t', pp.clean_text(text),'\n')

त्यांना 	--->	 त्यांना  

CO2 	--->	 co2  

सीओ२ 	--->	 सीओ२  

लागेल99 	--->	 लागेल #N  

Animalत्यांना 	--->	 animal त्यांना  

त्यांनाAnimal 	--->	 त्यांना animal  

Analogy_त्यांना 	--->	 analogy त्यांना  

Science२१ 	--->	 science #N  

१२Number 	--->	 #N number  

!@)$&%!#)&$!&$!$ 	--->	  

११.२२ 	--->	 #N  

१ 	--->	 #N  

1 	--->	 #N  

11.22 	--->	 #N  

a 	--->	  

B 	--->	  

सी 	--->	 सी  

